# What drives the price of a car?

![](images/kurt.jpeg)

**OVERVIEW**

In this application, you will explore a dataset from Kaggle. The original dataset contained information on 3 million used cars. The provided dataset contains information on 426K cars to ensure speed of processing.  Your goal is to understand what factors make a car more or less expensive.  As a result of your analysis, you should provide clear recommendations to your client -- a used car dealership -- as to what consumers value in a used car.

### CRISP-DM Framework

<center>
    <img src = images/crisp.png width = 50%/>
</center>


To frame the task, throughout our practical applications, we will refer back to a standard process in industry for data projects called CRISP-DM.  This process provides a framework for working through a data problem.  Your first step in this application will be to read through a brief overview of CRISP-DM [here](https://mo-pcco.s3.us-east-1.amazonaws.com/BH-PCMLAI/module_11/readings_starter.zip).  After reading the overview, answer the questions below.

### Business Understanding

From a business perspective, we are tasked with identifying key drivers for used car prices.  In the CRISP-DM overview, we are asked to convert this business framing to a data problem definition.  Using a few sentences, reframe the task as a data task with the appropriate technical vocabulary. 

**Determine the Business Objective**

The goal of this data analysis task is to determine what attributes of a car correlates well, or predicts the price of a car. This is a supervised learning exercise. We have the price of cars that were sold, along with features associated with the cars. Specifically, this is a regression problem.

We will divide the data set into two parts: a training and validation set. We will train a regression model on part of the training data, and validate the model on the other data set that the model has not seen.

**Data Understanding**

The data set provided, in general, includes features of cars, along with the price. That data is consistent with the goals of the exercise.

The data set is fairly large. There are over 400k samples provided. Some of that data may have missing values. Some columns may not be relevant. But at least the data appears to be compatible with the task.

To understand the data, we will look at features one at a time to get a sense of the type of data it contains, whether there are missing values and how we want to address that, and assess the descriptive statistics for that feature. We may also compare with other features to see if relationships emerge between the various features and the target. 

**Data Mining Goals**
  - Determine the features of cars that most influence the price of a vehicle.
  - For these features, identify what values leads to higher vehicle prices.
  - Determine how to model the relationships of the features, linear vs. polynomial features.
  - Determine what cross validation strategy to use to tune hyperparameters that yield optimized regression model


**Project Plan**
- Data Quality
  - Determine if there are redundant columns, like ID columns, that duplicate indexes or other information
  - Determine what data is missing
    - for each decide how to address, drop or fill in missing values
  - look for outliers & remove
  - look for invalid data
  - Determine how to model categorical columns
    - ordinal for values like condition that have some interpretation from better to worse, or bigger to smaller, or similar ordering.
    - One Hot encoding for values like manufacturer that have no such natural ordering interpretation

- Data Relationships
  - Correlations

- Data Preparation
  - Data Standardization
    - Ordinal Encoding
    - One Hot Encoding
  - Feature Engineering
    - Polynomial Transformation, if used
  - Data Normalization

- Data Modeling
  - Baseline model. Numerical values like odometer, year and price
  - Baseline model with polynomial features
  - Full model with categorical features
    - Linear regression
    - Ridge regression
    - Lasso regression
    - Ridge regression with Lasso Feature Selection

- Cross Validation
  - KFold cross validation to tune ```alpha``` values for Lasso estimator in feature selection, and Ridge Regression.

- Cluster Analysis
  - Visualize values of features that influence price.


### Data Understanding

After considering the business understanding, we want to get familiar with our data.  Write down some steps that you would take to get to know the dataset and identify any quality issues within.  Take time to get to know the dataset and explore what information it contains and how this could be used to inform your business understanding.

**Imports Needed by analysis**

In [15]:
# data utilities
import numpy as np
import pandas as pd
from datetime import datetime

# plotting utilities
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px

# machine learning utilities
from sklearn.pipeline import Pipeline, make_pipeline
from sklearn.impute import SimpleImputer
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.linear_model import LinearRegression, Ridge, Lasso
from sklearn.preprocessing import PolynomialFeatures, StandardScaler, OneHotEncoder, OrdinalEncoder
from sklearn.compose import make_column_transformer, ColumnTransformer
from sklearn.inspection import permutation_importance
from sklearn.feature_selection import SelectFromModel
from sklearn.metrics import mean_squared_error
from sklearn.decomposition import PCA
from sklearn.cluster import KMeans

import warnings
warnings.filterwarnings("ignore")

**Helper Functions**

In [16]:
# convenience function to evaluate data set during data analysis phase
def eval_categories(df):
  """
  Used in Exploratory Data Analysis to understand how the categorical data is distributed.
  For example, are there lots of nulls? A lot of unique values? Or a few set of values well distributed throughout the set.

  Args:
    df: Input data frame with categorical features (dtype = "object")

  Returns:
    a new data frame containing descriptive stats for each categorical feature in the input data frame
  """
  category_features = df.select_dtypes(include='object').columns.to_list()
  n = df.shape[0]
  to_percent = lambda x: f'{x/n:.1%}'
  return pd.DataFrame(
    {
      'unique': [len(np.unique(df[col].dropna())) for col in category_features],
      'top': [df.groupby(col)['price'].count().sort_values(ascending=False).head(1).index.to_list()[0] for col in category_features],
      'count': [to_percent(df.groupby(col)['price'].count().sort_values(ascending=False).head(1).values[0]) for col in category_features],
      'nulls': [to_percent(df[col].isnull().sum()) for col in category_features],
    },
    index=category_features
  )

**Load the data**

In [17]:
cars = pd.read_csv('data/vehicles.csv')
cars.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 426880 entries, 0 to 426879
Data columns (total 18 columns):
 #   Column        Non-Null Count   Dtype  
---  ------        --------------   -----  
 0   id            426880 non-null  int64  
 1   region        426880 non-null  object 
 2   price         426880 non-null  int64  
 3   year          425675 non-null  float64
 4   manufacturer  409234 non-null  object 
 5   model         421603 non-null  object 
 6   condition     252776 non-null  object 
 7   cylinders     249202 non-null  object 
 8   fuel          423867 non-null  object 
 9   odometer      422480 non-null  float64
 10  title_status  418638 non-null  object 
 11  transmission  424324 non-null  object 
 12  VIN           265838 non-null  object 
 13  drive         296313 non-null  object 
 14  size          120519 non-null  object 
 15  type          334022 non-null  object 
 16  paint_color   296677 non-null  object 
 17  state         426880 non-null  object 
dtypes: f

In [18]:
cars.sample(5)

,id,region,price,year,manufacturer,model,condition,cylinders,fuel,odometer,title_status,transmission,VIN,drive,size,type,paint_color,state
293665,7309699949,cleveland,7995,2013.0,kia,optima lx,excellent,4 cylinders,gas,121741.0,clean,automatic,NaN,fwd,mid-size,sedan,silver,oh
406333,7314935541,olympic peninsula,39999,2015.0,toyota,4runner,NaN,6 cylinders,gas,95542.0,clean,automatic,JTEBU5JR8F5261066,4wd,NaN,SUV,custom,wa
292069,7304392645,cincinnati,33275,2017.0,chevrolet,"silverado 1500 4wd double cab 143.5"" lt w/2lt",NaN,6 cylinders,NaN,32038.0,clean,automatic,1GCVKREH1HZ347962,4wd,NaN,pickup,black,oh
393811,7312297684,norfolk / hampton roads,21995,2011.0,ford,f150 fx4,NaN,NaN,gas,121438.0,clean,automatic,1FTFW1EF5BKD74252,NaN,NaN,other,white,va
205476,7316192090,lansing,16995,2011.0,ram,2500,good,8 cylinders,gas,123550.0,clean,automatic,3D7LT2ET3BG532916,4wd,full-size,truck,red,mi


In [19]:
eval_categories(cars).sort_values('unique', ascending=False)

,unique,top,count,nulls
VIN,118246,1FMJU1JT1HEA52352,0.1%,37.7%
model,29649,f-150,1.9%,1.2%
region,404,columbus,0.8%,0.0%
state,51,ca,11.9%,0.0%
manufacturer,42,ford,16.6%,4.1%
type,13,sedan,20.4%,21.8%
paint_color,12,white,18.6%,30.5%
cylinders,8,6 cylinders,22.1%,41.6%
title_status,6,clean,94.9%,1.9%
condition,6,good,28.5%,40.8%


**Observations**

- `id` and `VIN` are both identifiers and duplicate the index. They are not needed.
- `VIN`, `model`, and `region` have too many unique values to be useful. `VIN` should be unique, but values appear to be duplicated.
- 71.8% of `size` values are missing, too many for this feature to be used.

In [20]:
cars = cars.drop(columns=['id', 'VIN', 'model', 'region', 'size'])
eval_categories(cars).sort_values('unique', ascending=False)

,unique,top,count,nulls
state,51,ca,11.9%,0.0%
manufacturer,42,ford,16.6%,4.1%
type,13,sedan,20.4%,21.8%
paint_color,12,white,18.6%,30.5%
cylinders,8,6 cylinders,22.1%,41.6%
condition,6,good,28.5%,40.8%
title_status,6,clean,94.9%,1.9%
fuel,5,gas,83.4%,0.7%
transmission,3,automatic,78.8%,0.6%
drive,3,4wd,30.9%,30.6%


**Observations**
- `cylinders`, `condition`, `title_status` are ordinal features

In [21]:
cars[['cylinders_ordered', 'condition_ordered', 'title_status_ordered']] = OrdinalEncoder(dtype=float).fit_transform(cars[['cylinders', 'condition', 'title_status']])
eval_categories(cars[cars.columns.drop(['cylinders', 'condition', 'title_status'])]).sort_values('nulls', ascending=False)

,unique,top,count,nulls
manufacturer,42,ford,16.6%,4.1%
drive,3,4wd,30.9%,30.6%
paint_color,12,white,18.6%,30.5%
type,13,sedan,20.4%,21.8%
fuel,5,gas,83.4%,0.7%
transmission,3,automatic,78.8%,0.6%
state,51,ca,11.9%,0.0%


**Observations**
- `drive`, `paint_color`, and `type` are each missing 20% or more of their values. Remove the rows that are `null`.

In [22]:
cars = cars[(~cars['type'].isna())&(~cars['paint_color'].isna())&(~cars['drive'].isna())]
eval_categories(cars[cars.columns.drop(['cylinders', 'condition', 'title_status'])]).sort_values('count', ascending=False)

,unique,top,count,nulls
fuel,5,gas,85.3%,0.5%
transmission,3,automatic,80.4%,0.4%
drive,3,4wd,42.2%,0.0%
type,13,sedan,26.3%,0.0%
paint_color,12,white,25.5%,0.0%
manufacturer,42,ford,17.2%,3.6%
state,51,ca,11.4%,0.0%


**Observations**
- Data set has a few missing values that can be filled in with imputation.

In [23]:
cars.info()

<class 'pandas.core.frame.DataFrame'>
Index: 219285 entries, 31 to 426878
Data columns (total 16 columns):
 #   Column                Non-Null Count   Dtype  
---  ------                --------------   -----  
 0   price                 219285 non-null  int64  
 1   year                  218534 non-null  float64
 2   manufacturer          211298 non-null  object 
 3   condition             150855 non-null  object 
 4   cylinders             166024 non-null  object 
 5   fuel                  218161 non-null  object 
 6   odometer              217580 non-null  float64
 7   title_status          218447 non-null  object 
 8   transmission          218517 non-null  object 
 9   drive                 219285 non-null  object 
 10  type                  219285 non-null  object 
 11  paint_color           219285 non-null  object 
 12  state                 219285 non-null  object 
 13  cylinders_ordered     166024 non-null  float64
 14  condition_ordered     150855 non-null  float64
 15  titl

### Data Preparation

After our initial exploration and fine-tuning of the business understanding, it is time to construct our final dataset prior to modeling.  Here, we want to make sure to handle any integrity issues and cleaning, the engineering of new features, any transformations that we believe should happen (scaling, logarithms, normalization, etc.), and general preparation for modeling with `sklearn`. 

### Modeling

With your (almost?) final dataset in hand, it is now time to build some models.  Here, you should build a number of different regression models with the price as the target.  In building your models, you should explore different parameters and be sure to cross-validate your findings.

### Evaluation

With some modeling accomplished, we aim to reflect on what we identify as a high-quality model and what we are able to learn from this.  We should review our business objective and explore how well we can provide meaningful insight into drivers of used car prices.  Your goal now is to distill your findings and determine whether the earlier phases need revisitation and adjustment or if you have information of value to bring back to your client.

### Deployment

Now that we've settled on our models and findings, it is time to deliver the information to the client.  You should organize your work as a basic report that details your primary findings.  Keep in mind that your audience is a group of used car dealers interested in fine-tuning their inventory.